# 概要
* きれいに要約させてみる
* mode="summarize"を指定
* そもそも"。"で分割すればうまく行っていたのではないか？
* どうもindexの種類によって指定できるオプションが異なり、性能が異なる模様
* ListIndexならresponse_mode="tree_summarize"を指定でき、TreeIndexならmode="summarize"を指摘できる

# 参考
* イマイチ...
* https://gpt-index.readthedocs.io/en/latest/guides/use_cases.html#use-case-summarization-over-documents

In [1]:
import os
import openai

openai.api_key = os.getenv("OPENAI_API_KEY")

In [2]:
#Jupyterでも実行できるのは知らんかった...
import logging
import sys
logging.basicConfig(stream=sys.stdout, level=logging.DEBUG, force=True)

# PromptHelperとは
* トークンの制約に対応するための汎用プロンプトサポートツール

# LLM（Large Language Model）
* PromptHelperの引数の大部分はLLMに関連する
* つまり最初にLLMが何かを理解する必要がある
* (そもそも入力されたPromptをparseするのはLLMなのでPromptHelperがLLMを引数にとるのは当然である)
* 詳細はここを見よ！
* https://gpt-index.readthedocs.io/en/latest/reference/llm_predictor.html

# そもそも
* LLMPredictorとはLangchainのLLMChainのラッパーであり、LlamdaIndexと統合することができる
* LLMChain ↔ LLMPrecitor ↔ LlamdaIndex
* Our LLMPredictor is a wrapper around Langchain’s LLMChain that allows easy integration into LlamaIndex.
* llmを引数にとり、デフォルトではOpenAI’s text-davinci-003が指定される
* max_input_size: int = 3900, num_output: int = 256
* デフォルトでは入力は3900トークンで出力は256トークン
* つまり長文の回答はできない

# 引数のllm
* OpenAI以外にいも複数のLLMをサポートしている
* OpenAI, Cohere, Hugging Face, etc
* Llama_indexは自身でLLMはサポートしておらず、あくまでもlangchainを介して呼び出しを行う？

In [3]:
from gpt_index import LLMPredictor, GPTSimpleVectorIndex, SimpleDirectoryReader

In [4]:
# OpenAIのAPIのラッパー
from langchain.llms import OpenAI

# 利用例

In [12]:
from llama_index import (
    GPTSimpleVectorIndex, 
    GPTSimpleKeywordTableIndex, 
    GPTListIndex, 
    SimpleDirectoryReader,
    GPTTreeIndex
)

from llama_index import (
    GPTKeywordTableIndex, 
    SimpleDirectoryReader, 
    LLMPredictor,
    PromptHelper
)

from gpt_index.indices.knowledge_graph.base import GPTKnowledgeGraphIndex
from langchain import OpenAI

documents = SimpleDirectoryReader('story/dragonball').load_data()

# define prompt helper
# set maximum input size
max_input_size = 4096
# set number of output tokens
num_output = 1024
# set maximum chunk overlap
max_chunk_overlap = 0
# separatorが"/n/n"だとチャンクサイズの計算でエラーになるっぽい。"。"の場合は動く模様。
prompt_helper = PromptHelper(max_input_size, num_output, max_chunk_overlap, separator="。", chunk_size_limit=512)

# define LLM
llm_predictor = LLMPredictor(llm=OpenAI(temperature=0, model_name="text-davinci-003", max_tokens=num_output))

# build index
#index = GPTListIndex(documents, prompt_helper=prompt_helper, llm_predictor=llm_predictor)
index = GPTTreeIndex(documents, prompt_helper=prompt_helper, llm_predictor=llm_predictor)

DEBUG:root:> [SimpleDirectoryReader] Total files added: 1
INFO:root:> Building index from nodes: 1 chunks
DEBUG:openai:message='Request to OpenAI API' method=post path=https://api.openai.com/v1/completions
DEBUG:openai:api_version=None data='{"prompt": ["Write a summary of the following. Try to use only the information provided. Try to include as many key details as possible.\\n\\n\\n\\u5b6b\\u609f\\u7a7a\\u5c11\\u5e74\\u7de8\\n\\u5730\\u7403\\u306e\\u4eba\\u91cc\\u96e2\\u308c\\u305f\\u5c71\\u5965\\u306b\\u4f4f\\u3080\\u5c3b\\u5c3e\\u306e\\u751f\\u3048\\u305f\\u5c11\\u5e74\\u30fb\\u5b6b\\u609f\\u7a7a\\u306f\\u3042\\u308b\\u65e5\\u3001\\u897f\\u306e\\u90fd\\u304b\\u3089\\u3084\\u3063\\u3066\\u6765\\u305f\\u5c11\\u5973\\u30fb\\u30d6\\u30eb\\u30de\\u3068\\u51fa\\u4f1a\\u3046\\u3002\\u305d\\u3053\\u3067\\u30017\\u3064\\u96c6\\u3081\\u308b\\u3068\\u795e\\u9f8d\\uff08\\u30b7\\u30a7\\u30f3\\u30ed\\u30f3\\uff09\\u304c\\u73fe\\u308c\\u3001\\u3069\\u3093\\u306a\\u9858\\u3044\\u3067\\u3082\\u4e00

DEBUG:urllib3.connectionpool:Resetting dropped connection: api.openai.com
DEBUG:urllib3.connectionpool:https://api.openai.com:443 "POST /v1/completions HTTP/1.1" 200 None
DEBUG:openai:message='OpenAI API response' path=https://api.openai.com/v1/completions processing_ms=36053 request_id=0402a1860be59556883479f7010a93c4 response_code=200
DEBUG:root:
孫悟空少年編では、尻尾の生えた少年・孫悟空が西の都からやってきた少女・ブルマと出会い、ドラゴンボールを7つ集めると神龍（シェンロン）が現れるということを知り、残りのドラゴンボールを探す旅に出る。その後、悟空は武術の達人・亀仙人の下で修行を積み、天下一武道会に出場し準優勝を果たす。3年後の天下一武道会では、鶴仙流の天津飯と激突し準優勝に終わる。

ピッコロ大魔王編では、ピラフ一味によって復活したピッコロ大魔王によって、悟空の仲間たちや多くの武道家たちが殺される。悟空は仇を討つため、ピッコロと手を組み、自らの命と引き換えにラディッツを倒すが、約1年後にはさらに強力なサイヤ人たちがドラゴンボールを求めて地球に来襲することを知る。悟空は修行により増した力でナッパを一蹴し、ベジータと決闘し辛勝し撤退させる。

サイヤ人編では、実兄・ラディッツが宇宙より来襲し、自分が惑星ベジータの戦闘民族・サイヤ人であることを知らされる。悟空は宿敵ピッコロと手を組み、自らの命と引き換えにラディッツを倒すが、約1年後にはさらに強力なサイヤ人たちがドラゴンボールを求めて地球に来襲することを知る。悟空は修行により増した力でナッパを一蹴し、ベジータと決闘し辛勝し撤退させる。

フリーザ編では、悟空が重傷で入院中のため、悟飯、クリリン、ブルマの3人が神とピッコロの故郷であるナメック星へ向かう。そこでは宇宙の帝王・フリーザとその一味が不老不死を求めて来襲し、ドラゴンボールを略奪していた。悟飯たちはベジータ、フリーザ一

DEBUG:urllib3.connectionpool:https://api.openai.com:443 "POST /v1/completions HTTP/1.1" 200 None
DEBUG:openai:message='OpenAI API response' path=https://api.openai.com/v1/completions processing_ms=27155 request_id=8d3f1d0bc4d518a6b2cb717a2f1d80f7 response_code=200
DEBUG:root:
ドラゴンボールのストーリーは、悟空が怒りを爆発させて伝説の戦士・超サイヤ人へと覚醒し、フリーザを倒した後、ポルンガによって地球に帰還した悟飯たちが復活したドラゴンボールによりサイヤ人やフリーザ一味に殺された人々を蘇生させるところから始まる。続いて、未来からやってきたブルマとベジータの息子であるトランクスが悟空に未来の絶望を告げ、悟空が心臓病で危篤状態になるも特効薬で生還し、3年間各々に修行してセルゲームに臨む。セルゲームでは悟空が命を落とし、悟飯が超サイヤ人2へと覚醒してセルを撃破した。7年後、悟空が天下一武道会に出場するために占いババの力によって1日だけこの世に戻るが、その間に界王神から恐ろしい力を持つ魔人ブウの封印が解かれようとしていることを知らされる。魔人ブウにより悟飯やベジータが倒され、悟空はあの世に帰ったため、地球の命運は悟空の次男・孫悟天と少年トランクスの幼い二人に託される。悟空とベジータが界王神界で魔人ブウとの最終決戦に臨み、ドラゴンボールの協力もあり、地球・ナメック星・あの世の人々のエネルギーによって作り上げられた超特大の元気玉によって魔人ブウは完全に消滅する。10年後、悟空は孫のパンと共に天下一武道会に出場し、魔人ブウの生まれ変わりである少年・ウーブと出会い、修行に旅立ち、物語は幕を閉じる。
DEBUG:root:> 0/19, summary: 
孫悟空少年編では、尻尾の生えた少年・孫悟空が西の都からやってきた少女・ブルマと出会い、ドラゴ...
DEBUG:root:> 10/19, summary: 
ドラゴンボールのストーリーは、悟空が怒りを爆発させて伝説の戦士・超サイヤ人へと覚醒し、フリー...


In [13]:
summary = index.query("詳しく要約してください", mode="summarize")

INFO:root:> Starting query: 詳しく要約してください
DEBUG:root:> Searching in chunk: 孫悟空少年編
地球の人里離れた山奥に住む尻尾の生えた少年・孫悟空はある日、西の都からやって来た...
DEBUG:root:> Searching in chunk: 人さらいのウーロンや盗賊のヤムチャらを巻き込んだボール探しの末、世界征服を企むピラフ一味にボー...
DEBUG:root:> Searching in chunk: 

その後、悟空は旅の途中に知り合った武術の達人・亀仙人の下で、後に親友となるクリリンと共に8...
DEBUG:root:> Searching in chunk: さらに3年後の天下一武道会では、亀仙流のライバルである鶴仙流の天津飯（てんしんはん）と闘うが、...
DEBUG:root:> Searching in chunk: 悟空は仇を討つため、道中に出会ったヤジロベーや仙猫カリンの協力を得て命を賭して潜在する力を引き...
DEBUG:root:> Searching in chunk: 試合では、少年時代に出会った際に「嫁にもらう」と約束していた牛魔王の娘・チチと再会しその場で結...
DEBUG:root:> Searching in chunk: 

サイヤ人編
ピッコロ（マジュニア）との闘いから約5年後、息子の孫悟飯を儲けて平和な日々を過...
DEBUG:root:> Searching in chunk: 

悟空はドラゴンボールによって生き返るまでの間、あの世の界王の下で修業し、仲間と共に地球に強...
DEBUG:root:> Searching in chunk: 

フリーザ編
地球の神と殺された仲間たちを甦らせるため、重傷で入院中の悟空に代わり、悟飯、ク...
DEBUG:root:> Searching in chunk: 悟飯たちはベジータ、フリーザ一味とのドラゴンボールをめぐる三つ巴の攻防の末、後から到着した悟空...
DEBUG:root:> Searching in chunk: 怒りを爆発させた悟空は伝説の戦士・超（スーパー）サイヤ人へと覚醒。フルパワーを解放したフリーザ...
DEBUG:root:> Searching in chunk: 

人

DEBUG:urllib3.connectionpool:https://api.openai.com:443 "POST /v1/completions HTTP/1.1" 200 None
DEBUG:openai:message='OpenAI API response' path=https://api.openai.com/v1/completions processing_ms=34868 request_id=407f52477d94bd205d19db5e723116d1 response_code=200
DEBUG:root:
孫悟空少年編では、孫悟空とブルマがドラゴンボールを探して旅をする中で、ウーロンがとっさに言い放った下らない願いを叶えてもらうことで一味の野望を阻止する。その後、悟空は武術の達人・亀仙人の下で修行を積み、天下一武道会に出場し準優勝を果たす。3年後の天下一武道会では、鶴仙流の天津飯と闘い、準優勝に終わる。

ピッコロ大魔王編では、ピラフ一味によって復活したピッコロ大魔王によって、悟空の仲間たちや多くの武道家たちが殺される。悟空は仇を討つため、ピッコロと手を組み、自らの命と引き換えにラディッツを倒すが、約1年後にはさらに強力なサイヤ人たちがドラゴンボールを求めて地球に来襲することを知る。

サイヤ人編では、実兄・ラディッツが宇宙より来襲し、自分が惑星ベジータの戦闘民族・サイヤ人であることを知らされる。悟空は修行により増した力でナッパを一蹴し、ベジータと決闘し、何とか辛勝し撤退させるが、多くの仲間を失うとともに、ピッコロの戦死により彼と一心同体であった神も死亡し、地球のドラゴンボールも消滅する。

フリーザ編では、悟空が重傷で入院中のため、悟飯、クリリン、ブルマの3人が神とピッコロの故郷であるナメック星へ向かう。そこでは、界王すら畏怖する宇宙の帝王・フリーザとその一味が不老不死を求めて来襲し、ナメック星人を虐殺しながらドラゴンボールを略奪していた。悟飯たちはベジータ、フリーザ一味とのドラゴンボールをめぐる三つ巴の攻防の末、後から到着した悟空とナメック星人たちの協力を得てナメック星の神龍・ポルンガを呼び出し、ピッコロと地球のドラゴンボールを復活させる。最後に悟空は、チチと結婚し、マジュニアと決勝戦で激突し、初の天下一武道会優勝を飾る

DEBUG:urllib3.connectionpool:https://api.openai.com:443 "POST /v1/completions HTTP/1.1" 200 None
DEBUG:openai:message='OpenAI API response' path=https://api.openai.com/v1/completions processing_ms=19760 request_id=ac59f43cb989ba204f42b3af40624e7c response_code=200
DEBUG:root:
ドラゴンボールのストーリーは、悟空と仲間たちがフリーザやポルンガ、セル、魔人ブウといった強力な敵と戦い、地球を守るために命を落とす場面がある。悟空は自分が悪人を引き寄せているという理由で生き返りを拒否し、あの世に残ることを選ぶ。セルゲームを制した悟飯たちは、ドラゴンボールによりセルに殺された人々を蘇生させる。魔人ブウが復活した際、悟空はあの世に帰ったため、地球の命運は悟空の次男・孫悟天と少年トランクスの幼い二人に託される。悟空とベジータが界王神界で真の姿となった魔人ブウとの最終決戦に臨み、ドラゴンボールの協力もあり、地球・ナメック星・あの世の人々のエネルギーによって作り上げられた超特大の元気玉によって魔人ブウは完全に消滅する。それから10年後、悟空は孫のパンと共に天下一武道会に久しぶりに出場し、魔人ブウの生まれ変わりである少年・ウーブと出会う。悟空はウーブと共に見果てぬ強さを追い求めて修行に旅立ち、物語は幕を閉じる。
DEBUG:openai:message='Request to OpenAI API' method=post path=https://api.openai.com/v1/completions
DEBUG:openai:api_version=None data='{"prompt": ["Context information is below. \\n---------------------\\n\\n\\n\\n\\n\\n\\u5b6b\\u609f\\u7a7a\\u5c11\\u5e74\\u7de8\\u3067\\u306f\\u3001\\u5c3b\\u5c3e\\u306e\\u

DEBUG:urllib3.connectionpool:https://api.openai.com:443 "POST /v1/completions HTTP/1.1" 200 None
DEBUG:openai:message='OpenAI API response' path=https://api.openai.com/v1/completions processing_ms=35788 request_id=d085d26bd769e39c8e81132abd278119 response_code=200
DEBUG:root:
ドラゴンボールのストーリーは、悟空が西の都からやってきた少女・ブルマと出会い、ドラゴンボールを7つ集めると神龍（シェンロン）が現れるということを知り、残りのドラゴンボールを探す旅に出ることから始まる。ピラフ一味によって復活したピッコロ大魔王によって、悟空の仲間たちや多くの武道家たちが殺される。悟空は仇を討つため、ピッコロと手を組み、自らの命と引き換えにラディッツを倒す。実兄・ラディッツが宇宙より来襲し、自分が惑星ベジータの戦闘民族・サイヤ人であることを知らされる。悟空は宿敵ピッコロと手を組み、自らの命と引き換えにラディッツを倒す。悟空が重傷で入院中のため、悟飯、クリリン、ブルマの3人が神とピッコロの故郷であるナメック星へ向かう。そこでは宇宙の帝王・フリーザとその一味が不老不死を求めて来襲し、ドラゴンボールを略奪していた。悟飯たちはベジータ、フリーザ一味とのドラゴンボールをめぐる三つ巴の攻防の末、悟空とナメック星人たちの協力を得てポルンガを呼び出し、ピッコロと地球のドラゴンボールを復活させる。悟空が怒りを爆発させて伝説の戦士・超サイヤ人へと覚醒し、フリーザを倒した後、ポルンガによって地球に帰還した悟飯たちが復活したドラゴンボールによりサイヤ人やフリーザ一味に殺された人々を蘇生させる。未来からやってきたブルマとベジータの息子であるトランクスが悟空に未来の絶望を告げ、悟空が心臓病で危篤状態になるも特効薬で生還し、3年間各々に修行してセルゲームに臨む。セルゲームでは悟空が命を落とし、悟飯が超サイヤ人2へと覚醒してセルを撃破した。界王神から恐ろしい力を持つ魔人ブウの封印が解かれようとしていることを知らされる。魔人ブウにより悟飯やベジ

DEBUG:urllib3.connectionpool:https://api.openai.com:443 "POST /v1/completions HTTP/1.1" 200 None
DEBUG:openai:message='OpenAI API response' path=https://api.openai.com/v1/completions processing_ms=26317 request_id=5c77e51961c9add00f09efe9d8a8b143 response_code=200
DEBUG:root:

孫悟空少年編では、孫悟空とブルマがドラゴンボールを探して旅をする中で、ウーロンがとっさに言い放った下らない願いを叶えてもらうことで一味の野望を阻止する。その後、悟空は武術の達人・亀仙人の下で修行を積み、天下一武道会に出場し準優勝を果たす。ピッコロ大魔王編では、ピラフ一味によって復活したピッコロ大魔王によって、悟空の仲間たちや多くの武道家たちが殺される。悟空は仇を討つため、ピッコロと手を組み、自らの命と引き換えにラディッツを倒すが、約1年後にはさらに強力なサイヤ人たちがドラゴンボールを求めて地球に来襲することを知る。サイヤ人編では、実兄・ラディッツが宇宙より来襲し、自分が惑星ベジータの戦闘民族・サイヤ人であることを知らされる。悟空と仲間たちはフリーザやポルンガ、セル、魔人ブウといった強力な敵と戦い、地球を守るために命を落とす場面がある。悟空は自分が悪人を引き寄せているという理由で生き返りを拒否し、あの世に残ることを選ぶ。セルゲームを制した悟飯たちは、ドラゴンボールによりセルに殺された人々を蘇生させる。魔人ブウが復活した際、悟空はあの世に帰ったため、地球の命運は悟空の次男・孫悟天と少年トランクスの幼い二人に託される。
DEBUG:root:> Refined response: 

孫悟空少年編では、孫悟空とブルマがドラゴンボールを探して旅をする中で、ウーロンがとっさに言い放った下らない願いを叶えてもらうことで一味の野望を阻止する。その後、悟空は武術の達人・亀仙人の下で修行を積み、天下一武道会に出場し準優勝を果たす。ピッコロ大魔王編では、ピラフ一味によって復活したピッコロ大魔王によって、悟空の仲間たちや多くの武道家たちが殺さ

DEBUG:urllib3.connectionpool:https://api.openai.com:443 "POST /v1/completions HTTP/1.1" 200 None
DEBUG:openai:message='OpenAI API response' path=https://api.openai.com/v1/completions processing_ms=29038 request_id=8e5c9679381461ec0038ef2ba994386d response_code=200
DEBUG:root:

孫悟空少年編では、孫悟空とブルマがドラゴンボールを探して旅をする中で、ウーロンがとっさに言い放った下らない願いを叶えてもらうことで一味の野望を阻止する。その後、悟空は武術の達人・亀仙人の下で修行を積み、天下一武道会に出場し準優勝を果たす。ピッコロ大魔王編では、ピラフ一味によって復活したピッコロ大魔王によって、悟空の仲間たちや多くの武道家たちが殺される。悟空は仇を討つため、ピッコロと手を組み、自らの命と引き換えにラディッツを倒すが、約1年後にはさらに強力なサイヤ人たちがドラゴンボールを求めて地球に来襲することを知る。サイヤ人編では、実兄・ラディッツが宇宙より来襲し、自分が惑星ベジータの戦闘民族・サイヤ人であることを知らされる。悟空と仲間たちはフリーザやポルンガ、セル、魔人ブウといった強力な敵と戦い、地球を守るために命を落とす場面がある。悟空は自分が悪人を引き寄せているという理由で生き返りを拒否し、あの世に残ることを選ぶ。界王神界で真の姿となった魔人ブウとの最終決戦に臨み、ドラゴンボールの協力もあり、地球・ナメック星・あの世の人々のエネルギーによって作り上げられた超特大の元気玉によって魔人ブウは完全に消滅する。セルゲームを制した悟飯たちは、ドラゴンボールによりセルに殺された人々を蘇生させる。魔人ブウが復活した際、悟空はあの世に帰ったため、地球の命運は悟空の次男・孫悟天と少年トランクスの幼い二人に託される。
DEBUG:root:> Refined response: 

孫悟空少年編では、孫悟空とブルマがドラゴンボールを探して旅をする中で、ウーロンがとっさに言い放った下らない願いを叶えてもらうことで一味の野望を阻止する。その

DEBUG:root:> Refined response: 

孫悟空少年編では、孫悟空とブルマがドラゴンボールを探して旅をする中で、ウーロンがとっさに言い放った下らない願いを叶えてもらうことで一味の野望を阻止する。その後、悟空は武術の達人・亀仙人の下で修行を積み、天下一武道会に出場し準優勝を果たす。ピッコロ大魔王編では、ピラフ一味によって復活したピッコロ大魔王によって、悟空の仲間たちや多くの武道家たちが殺される。悟空は仇を討つため、ピッコロと手を組み、自らの命と引き換えにラディッツを倒すが、約1年後にはさらに強力な宇宙の帝王・フリーザとその一味が不老不死を求めて来襲し、ドラゴンボールを略奪していることを知る。悟空が重傷で入院中のため、悟飯、クリリン、ブルマの3人が神とピッコロの故郷であるナメック星へ向かう。サイヤ人編では、実兄・ラディッツが宇宙より来襲し、自分が惑星ベジータの戦闘民族・サイヤ人であることを知らされる。悟空と仲間たちはフリーザやポルンガ、セル、魔人ブウといった強力な敵と戦い、地球を守るために命を落とす場面がある。悟空は自分が悪人を引き寄せているという理由で生き返りを拒否し、あの世に残ることを選ぶ。界王神界で真の姿となった魔人ブウとの最終決戦に臨み、ドラゴンボールの協力もあり、地球・ナメック星・あの世の人々のエネルギーによって作り上げられた超特大の元気玉によって魔人ブウは完全に消滅する。セルゲームを制した悟飯たちは、ドラゴンボールによりセルに殺された人々を蘇生させる。魔人ブウが復活した際、悟空はあの世に帰ったため、地球の命運は悟空の次男・孫悟天と少年トランクスの幼い二人に託される。
INFO:root:> [query] Total LLM token usage: 16164 tokens
INFO:root:> [query] Total embedding token usage: 0 tokens


In [14]:
summary

Response(response='\n\n孫悟空少年編では、孫悟空とブルマがドラゴンボールを探して旅をする中で、ウーロンがとっさに言い放った下らない願いを叶えてもらうことで一味の野望を阻止する。その後、悟空は武術の達人・亀仙人の下で修行を積み、天下一武道会に出場し準優勝を果たす。ピッコロ大魔王編では、ピラフ一味によって復活したピッコロ大魔王によって、悟空の仲間たちや多くの武道家たちが殺される。悟空は仇を討つため、ピッコロと手を組み、自らの命と引き換えにラディッツを倒すが、約1年後にはさらに強力な宇宙の帝王・フリーザとその一味が不老不死を求めて来襲し、ドラゴンボールを略奪していることを知る。悟空が重傷で入院中のため、悟飯、クリリン、ブルマの3人が神とピッコロの故郷であるナメック星へ向かう。サイヤ人編では、実兄・ラディッツが宇宙より来襲し、自分が惑星ベジータの戦闘民族・サイヤ人であることを知らされる。悟空と仲間たちはフリーザやポルンガ、セル、魔人ブウといった強力な敵と戦い、地球を守るために命を落とす場面がある。悟空は自分が悪人を引き寄せているという理由で生き返りを拒否し、あの世に残ることを選ぶ。界王神界で真の姿となった魔人ブウとの最終決戦に臨み、ドラゴンボールの協力もあり、地球・ナメック星・あの世の人々のエネルギーによって作り上げられた超特大の元気玉によって魔人ブウは完全に消滅する。セルゲームを制した悟飯たちは、ドラゴンボールによりセルに殺された人々を蘇生させる。魔人ブウが復活した際、悟空はあの世に帰ったため、地球の命運は悟空の次男・孫悟天と少年トランクスの幼い二人に託される。', source_nodes=[SourceNode(source_text='孫悟空少年編\n地球の人里離れた山奥に住む尻尾の生えた少年・孫悟空はある日、西の都からやって来た少女・ブルマと出会う。そこで、7つ集めると神龍（シェンロン）が現れ、どんな願いでも一つだけ叶えてくれるというドラゴンボールの存在を、さらに育ての親である孫悟飯の形見として大切に持っていた球がその1つ「四星球（スーシンチュウ）」であることを知り、ブルマと共に残りのドラゴンボールを探す旅に出る', doc_id='ba5ba915-9865-4de4-9bad-b952522b7655', extra_info=None,

In [15]:
summary.source_nodes

[SourceNode(source_text='孫悟空少年編\n地球の人里離れた山奥に住む尻尾の生えた少年・孫悟空はある日、西の都からやって来た少女・ブルマと出会う。そこで、7つ集めると神龍（シェンロン）が現れ、どんな願いでも一つだけ叶えてくれるというドラゴンボールの存在を、さらに育ての親である孫悟飯の形見として大切に持っていた球がその1つ「四星球（スーシンチュウ）」であることを知り、ブルマと共に残りのドラゴンボールを探す旅に出る', doc_id='ba5ba915-9865-4de4-9bad-b952522b7655', extra_info=None, node_info=None, similarity=None),
 SourceNode(source_text='人さらいのウーロンや盗賊のヤムチャらを巻き込んだボール探しの末、世界征服を企むピラフ一味にボールを奪われ神龍を呼び出されるが、ウーロンがとっさに言い放った下らない願いを叶えてもらうことで一味の野望を阻止する', doc_id='ba5ba915-9865-4de4-9bad-b952522b7655', extra_info=None, node_info=None, similarity=None),
 SourceNode(source_text='\n\nその後、悟空は旅の途中に知り合った武術の達人・亀仙人の下で、後に親友となるクリリンと共に8か月間にわたる修行を積み、その成果を確かめるために世界一の武術の達人を決める天下一武道会に出場し、変装して出場していた亀仙人に敗れるも準優勝を果たす。悟空は再び修行の旅へと出発し、ドラゴンボールの悪用を企むレッドリボン軍との闘いや、孫悟飯との再会などを経てさらに強さを増していく', doc_id='ba5ba915-9865-4de4-9bad-b952522b7655', extra_info=None, node_info=None, similarity=None),
 SourceNode(source_text='さらに3年後の天下一武道会では、亀仙流のライバルである鶴仙流の天津飯（てんしんはん）と闘うが、あと一歩のところで敗れ、前回と同じく準優勝に終わる。\n\nピッコロ大魔王編\n天下一武道会終了後、ピラフ一味によって復活したピッコ